In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import LeakyReLU, CuDNNLSTM, CuDNNGRU
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

features = np.load('../data/features.npy')
labels = np.load('../data/labels.npy', allow_pickle=True)

test_indices = [np.load('../data/test_index1.npy'),
                np.load('../data/test_index2.npy'),
                np.load('../data/test_index3.npy'),
                np.load('../data/test_index4.npy'),
                np.load('../data/test_index5.npy')]

extra_benign = np.load('../data/extra_benign.npy')

models = {'mlp1':[200, 200, 10], 'mlp2':[400, 400, 20],'mlp3': [1200, 600, 20],
          'mlp4':[200, 200, 200, 20], 'mlp5':[200, 400, 400, 20], 'mlp6':[400, 800, 200, 20],
          'lstm1':[30], 'lstm2':[30, 30],'lstm3': [30, 30, 30],
          'lstm4':[60], 'lstm5':[60, 60], 'lstm6':[60, 60, 60],
          'gru1':[30], 'gru2':[30, 30],'gru3': [30, 30, 30],
          'gru4':[60], 'gru5':[60, 60], 'gru6':[60, 60, 60]}

models_folder = 'Models'
if not os.path.exists(models_folder): os.mkdir(models_folder)

for model_name, model_layers in models.items():
    for fold in range(1,6):
        output_folder = os.path.join(models_folder, model_name + '_' + str(fold))
        model_file_path = os.path.join(output_folder, 'model.json')
        weight_file_path = os.path.join(output_folder, 'weights.h5')
        if os.path.exists(model_file_path): 
            print(model_name, 'for fold', fold, 'exists. Skipping to the next model...')
            continue
        if not os.path.exists(output_folder): os.mkdir(output_folder)
        
        mask = np.ones(len(labels), dtype=bool)
        mask[test_indices[fold-1],] = False
        mask[extra_benign,] = False; # commented out when training with complete (test_all and test_uniform are same.)
        x_train = features[mask]
        y_train = labels[mask]
        
        minmax_scaler = preprocessing.MinMaxScaler()
        x_train = minmax_scaler.fit_transform(x_train)
        
        label_encoder = preprocessing.LabelEncoder()
        y_train = label_encoder.fit_transform(y_train)
        
        one_hot_encoder = preprocessing.OneHotEncoder(sparse=False)
        y_train = one_hot_encoder.fit_transform(y_train.reshape(-1,1))
        
        if not model_name.startswith('mlp'):
            x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
            input_shape = (x_train.shape[1], x_train.shape[2])
        else:
            input_dim = x_train.shape[1] 
        
        x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,
                                                          test_size=0.25, random_state=42)
       
        class_number = y_train.shape[1]
        
        print('Creating', model_name, 'for fold', fold, '...')
        model = Sequential()
        for layer_size in model_layers:
            if model_name.startswith('mlp'):
                if len(model.layers) == 0:
                    model.add(Dense(layer_size, input_dim=input_dim, kernel_initializer='normal', activation='linear'))
                else:
                    model.add(Dense(layer_size, kernel_initializer='normal', activation='linear'))
                model.add(LeakyReLU(alpha=0.01))
            elif model_name.startswith('lstm'):
                if len(model.layers) == 0: 
                    model.add(CuDNNLSTM(layer_size, input_shape=input_shape, return_sequences=(len(model.layers) != len(model_layers)-1)))
                else:
                    model.add(CuDNNLSTM(layer_size, return_sequences=(len(model.layers) != len(model_layers)-1)))
            else:
                if len(model.layers) == 0: 
                    model.add(CuDNNGRU(layer_size, input_shape=input_shape, return_sequences=(len(model.layers) != len(model_layers)-1)))
                else:
                    model.add(CuDNNGRU(layer_size, return_sequences=(len(model.layers) != len(model_layers)-1)))
        model.add(Dense(class_number, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model.summary()
        
        checkpoint_path = os.path.join(output_folder, 'weight.E{epoch:02d}.h5')
        checkpointer = ModelCheckpoint(filepath=checkpoint_path, verbose=0)
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3,
                                patience=5, verbose=2, mode='auto')
        
        start_epoch = 0
        for file in os.listdir(output_folder):
            if file.endswith('.h5'): start_epoch += 1
        
        if start_epoch > 0:
            model.load_weights(os.path.join(output_folder, file))
            print('Training resumes from epoch #', start_epoch)
        else:
            print('Training starts from scratch...')
        
        history = model.fit(x_train, y_train, validation_data=(x_val,y_val),
                            callbacks=[checkpointer,monitor], initial_epoch=start_epoch,
                            verbose=2, epochs=100)
        
        print('Training completed. Saving the model and the final weights...')
        with open(model_file_path, "w") as json_file: json_file.write(model.to_json())
        model.save_weights(weight_file_path)
        
        print(history.history)
        # plot and save history for accuracy
        plt.figure()
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['training', 'validation'], loc='upper left')
        plt.savefig(os.path.join(output_folder, 'accuracy.png'))
        # plot and save history for loss
        plt.figure()
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['training', 'validation'], loc='upper right')
        plt.savefig(os.path.join(output_folder, 'loss.png'))
        
        del x_train, y_train, model